#### ReportLab use guide

https://www.reportlab.com/docs/reportlab-userguide.pdf

#### ReportLab source code

https://bitbucket.org/rptlab/reportlab/src/998faa6891c038a2453a7828df2be53f7b68ce1d/src/reportlab/?at=default

#### Notes

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

https://pairlist2.pair.net/pipermail/reportlab-users/2005-June/004070.html

* Very useful code snippet: 

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

* Reportlab: Mixing Fixed Content and Flowables

https://www.blog.pythonlibrary.org/2012/06/27/reportlab-mixing-fixed-content-and-flowables/



## Approach

Approaches to generating calendar pages:

a) Don't use page templates, use just canvas and page breaks

It's possible to draw table on canvas! That's good.

b) Use templates and implement custom flowables to allow custom drawing in the pages

https://www.blog.pythonlibrary.org/2014/03/10/reportlab-how-to-create-custom-flowables/

c) Not sure is possible: use templates and somehow draw on canvas on each page.

Try to somehow access the canvas while creating the story. Maybe some get_canvas method.

d) Use page templates and use global variable to change what's on each page - 

```
def on_page3(canvas, doc):
    global page_data
    canvas.saveState()
    canvas.line(20,20, -20,20)
    canvas.restoreState()
    canvas.drawString(100, 600, f"String on page template 3:{page_data}")
```

I will probably have to use a combination of frames and canvas, because I need tables and tables are flowables. So maybe put a table tightly into a frame. 

## Notes
#### Grid instead of table:
canas.grid(), page 16 of ReportLab manual 

Advantage: it's not flowable, can place it exactly
Disadvantage: how do I put string in the cells? ... tedious

In [ ]:
from reportlab.lib.units import inch, cm, mm
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, B5
from reportlab.platypus import SimpleDocTemplate, BaseDocTemplate, PageTemplate, Frame, Spacer, Table, \
TableStyle, Paragraph, NextPageTemplate
from reportlab.graphics.shapes import Drawing
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfgen import canvas
from reportlab.lib.utils import isSeq, encode_label, decode_label, annotateException, strTypes
from reportlab.platypus import Table as pdfTable

import calendar
import datetime

import os
import logging
import copy
from collections import namedtuple


In [ ]:
import monthdelta

In [ ]:
import reportlab
import pandas as pd

In [4]:
logger = logging.Logger('root', level=logging.DEBUG)
logging.basicConfig()

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter.datefmt = '%m/%d/%Y %I:%M:%S %p' #03/11/2018 11:54:48 AM

#console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

logger.addHandler(console_handler)


In [5]:
class Options:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)
    def __str__(self):
        return "".join([ str(key)+":"+str(value)+"\n" for key, value in self.__dict__.items()])

    

In [42]:
def fill_options():
    
    debug_verbosity = 1
    
    layout=1 
    #format={
    #    'start_date' : datetime.date(2018,10,1),
    #    'end_date' : datetime.date(2019,6,4)
    #}
    daterange={
                'start_date' : datetime.date(2018,12,25),
                #'start_date' : format['start_date'],
                'end_date' : datetime.date(2019,1,10),
                #'end_date' : format['end_date']
        
              }
    monthrange={
                'start' : daterange['start_date'],
                'stop' : daterange['end_date']
                }
    # diary structure as a mapping pages->sections
    # Is generated from diary_structure_template
    # diary_structure_template -> diary_sections

    sections = ['info_sheet'] + ['months_v1'] + 4*['notes_v1'] + ['weeks_v2'] + 4*['notes_v1']

    pagesize = [t for t in reversed(reportlab.lib.pagesizes.B5)] # landscape
    margins = tuple(4*[0.5*cm]) #left, right, upper, lower

    return Options(
            debug_verbosity = debug_verbosity,
            layout=layout, 
            format=format, 
            daterange=daterange, 
            monthrange=monthrange, 
            sections=sections,
            pagesize=pagesize,
            margins=margins)


o = fill_options()

In [43]:
logger.info(o)

06/07/2018 08:56:02 PM - root - INFO - debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 12, 25), 'end_date': datetime.date(2019, 1, 10)}
monthrange:{'start': datetime.date(2018, 12, 25), 'stop': datetime.date(2019, 1, 10)}
sections:['info_sheet', 'months_v1', 'notes_v1', 'notes_v1', 'notes_v1', 'notes_v1', 'weeks_v2', 'notes_v1', 'notes_v1', 'notes_v1', 'notes_v1']
pagesize:[708.6614173228346, 500.3149606299212]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)



In [27]:
# global variable VERBOSITY
VERBOSITY = o.debug_verbosity

In [28]:
height = o.pagesize[1]
width = o.pagesize[0]

In [29]:
# fix this 
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [30]:
def nearest_past_Monday(initial_date):
# find the date of the nearet present or past Monday
    logger.debug(f'Initial date:{initial_date}, {weekdays[initial_date.weekday()]}')
    for i in range(7):
        # iterate 0 to 6
        previous = initial_date - datetime.timedelta(days=i)
        day = weekdays[previous.weekday()]
        logger.debug(f'{previous}, {day}')
        if day == 'Monday':
            return previous
        
    assert(False), "Something wrong. Iterated back 7 days and didn't find a Monday."
            
if 1:
    print(nearest_past_Monday( datetime.date(2018,5,8)))
    print(nearest_past_Monday( datetime.date(2018,5,7)))
    print(nearest_past_Monday( datetime.date(2018,5,6)))
    

06/07/2018 08:52:40 PM - root - DEBUG - Initial date:2018-05-08, Tuesday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-08, Tuesday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-07, Monday
06/07/2018 08:52:40 PM - root - DEBUG - Initial date:2018-05-07, Monday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-07, Monday
06/07/2018 08:52:40 PM - root - DEBUG - Initial date:2018-05-06, Sunday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-06, Sunday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-05, Saturday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-04, Friday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-03, Thursday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-02, Wednesday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-05-01, Tuesday
06/07/2018 08:52:40 PM - root - DEBUG - 2018-04-30, Monday


2018-05-07
2018-05-07
2018-04-30


In [32]:
def nearest_future_Sunday(initial_date):
# find the date of the nearet present or future Sunday
    logger.debug(f'Initial date:, {initial_date}, {weekdays[initial_date.weekday()]}')
    for i in range(7):
        # iterate 0 to 6
        next = initial_date + datetime.timedelta(days=i)
        day = weekdays[next.weekday()]
        logger.debug(f'{next},{day}')
        if day == 'Sunday':
            return next
        
    assert(False), "Something wrong. Iterated back 7 days and didn't find a Monday."
            
if 1:
    print(nearest_future_Sunday( datetime.date(2018,5,5)))
    print(nearest_future_Sunday( datetime.date(2018,5,6)))
    print(nearest_future_Sunday( datetime.date(2018,5,7)))

06/07/2018 08:52:47 PM - root - DEBUG - Initial date:, 2018-05-05, Saturday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-05,Saturday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-06,Sunday
06/07/2018 08:52:47 PM - root - DEBUG - Initial date:, 2018-05-06, Sunday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-06,Sunday
06/07/2018 08:52:47 PM - root - DEBUG - Initial date:, 2018-05-07, Monday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-07,Monday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-08,Tuesday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-09,Wednesday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-10,Thursday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-11,Friday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-12,Saturday
06/07/2018 08:52:47 PM - root - DEBUG - 2018-05-13,Sunday


2018-05-06
2018-05-06
2018-05-13


In [33]:
def get_calendar_boundaries(start_date, end_date):
    # return the date of the nearest previous Monday and the date of the nearest next Sunday in a tuple

    assert(start_date<=end_date), "Start day must be earlier or the same as end date"

    diary_first_date = nearest_past_Monday(start_date)
    diary_last_date = nearest_future_Sunday(end_date)

    # check it's at least 1 week
    assert( (diary_last_date - diary_first_date).days >= 6 )

    # check it's exactly an integer multiple of 7
    assert( ((diary_last_date - diary_first_date).days+1)//7 - ((diary_last_date - diary_first_date).days+1)/7 == 0 ), "It's not whole weeks"

    # check that the first day is Monday
    assert( diary_first_date.weekday()==0 ), "First date must be Monday"
    # check that the last day is Sunday
    assert( diary_last_date.weekday()==6 ), "Last date must be Sunday"
    
    return (diary_first_date, diary_last_date)

if 1:
    print(o)
    print( get_calendar_boundaries(o.daterange['start_date'], o.daterange['end_date'] ))

06/07/2018 08:52:49 PM - root - DEBUG - Initial date:2018-10-01, Monday
06/07/2018 08:52:49 PM - root - DEBUG - 2018-10-01, Monday
06/07/2018 08:52:49 PM - root - DEBUG - Initial date:, 2019-06-04, Tuesday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-04,Tuesday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-05,Wednesday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-06,Thursday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-07,Friday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-08,Saturday
06/07/2018 08:52:49 PM - root - DEBUG - 2019-06-09,Sunday


debug_verbosity:1
layout:1
format:{'start_date': datetime.date(2018, 10, 1), 'end_date': datetime.date(2019, 6, 4)}
daterange:{'start_date': datetime.date(2018, 10, 1), 'end_date': datetime.date(2019, 6, 4)}
monthrange:{'start': datetime.date(2018, 10, 1), 'stop': datetime.date(2019, 6, 4)}
sections:['info_sheet', 'months_v1', 'notes_v1', 'notes_v1', 'notes_v1', 'notes_v1', 'weeks_v2', 'notes_v1', 'notes_v1', 'notes_v1', 'notes_v1']
pagesize:[708.6614173228346, 500.3149606299212]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)

(datetime.date(2018, 10, 1), datetime.date(2019, 6, 9))


In [40]:
def get_days(options):
    '''Return a DataFrame with days for the weeks section'''
    
    # iterate through days of the diary
    (first_date, last_date) = \
        get_calendar_boundaries(options.daterange['start_date'], options.daterange['end_date'] )
    print(first_date, last_date)

    df_diary = pd.DataFrame()

    logger.debug("Days of the diary:")
    day = first_date
    week_index = 0
    i = 0
    while day<=last_date:
        df_diary = df_diary.append({'week_index': int(week_index), 'date': day, 'weekday': weekdays[day.weekday()]}, ignore_index=True)

        logger.debug(f'{week_index}, {day}, {weekdays[day.weekday()]}')
        day += datetime.timedelta(days=1)

        i += 1
        if i % 7 ==0:
            week_index += 1

    df_diary['week_index'] = df_diary['week_index'].astype(int)
    df_diary['week_number'] =  df_diary['date'].apply(lambda x: x.isocalendar()[1])
    df_diary['month_number'] =  df_diary['date'].apply(lambda x: x.month)
    df_diary['month_name'] = df_diary['date'].apply(lambda x: x.strftime("%B"))
    
    return df_diary

if 1:
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    df_days = get_days(options=test_options)
    display(df_days)

06/07/2018 08:53:35 PM - root - DEBUG - Initial date:2018-12-24, Monday
06/07/2018 08:53:35 PM - root - DEBUG - 2018-12-24, Monday
06/07/2018 08:53:35 PM - root - DEBUG - Initial date:, 2019-06-10, Monday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-10,Monday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-11,Tuesday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-12,Wednesday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-13,Thursday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-14,Friday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-15,Saturday
06/07/2018 08:53:35 PM - root - DEBUG - 2019-06-16,Sunday
06/07/2018 08:53:35 PM - root - DEBUG - Days of the diary:
06/07/2018 08:53:35 PM - root - DEBUG - 0, 2018-12-24, Monday
06/07/2018 08:53:35 PM - root - DEBUG - 0, 2018-12-25, Tuesday
06/07/2018 08:53:35 PM - root - DEBUG - 0, 2018-12-26, Wednesday
06/07/2018 08:53:35 PM - root - DEBUG - 0, 2018-12-27, Thursday
06/07/2018 08:53:35 PM - root - DEBUG - 0, 2018-12-28, Friday
06/07/

2018-12-24 2019-06-16


06/07/2018 08:53:35 PM - root - DEBUG - 3, 2019-01-20, Sunday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-21, Monday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-22, Tuesday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-23, Wednesday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-24, Thursday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-25, Friday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-26, Saturday
06/07/2018 08:53:35 PM - root - DEBUG - 4, 2019-01-27, Sunday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-01-28, Monday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-01-29, Tuesday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-01-30, Wednesday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-01-31, Thursday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-02-01, Friday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-02-02, Saturday
06/07/2018 08:53:35 PM - root - DEBUG - 5, 2019-02-03, Sunday
06/07/2018 08:53:35 PM - root - DEBUG - 6, 2019-02-04,

06/07/2018 08:53:36 PM - root - DEBUG - 22, 2019-05-29, Wednesday
06/07/2018 08:53:36 PM - root - DEBUG - 22, 2019-05-30, Thursday
06/07/2018 08:53:36 PM - root - DEBUG - 22, 2019-05-31, Friday
06/07/2018 08:53:36 PM - root - DEBUG - 22, 2019-06-01, Saturday
06/07/2018 08:53:36 PM - root - DEBUG - 22, 2019-06-02, Sunday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-03, Monday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-04, Tuesday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-05, Wednesday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-06, Thursday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-07, Friday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-08, Saturday
06/07/2018 08:53:36 PM - root - DEBUG - 23, 2019-06-09, Sunday
06/07/2018 08:53:36 PM - root - DEBUG - 24, 2019-06-10, Monday
06/07/2018 08:53:36 PM - root - DEBUG - 24, 2019-06-11, Tuesday
06/07/2018 08:53:36 PM - root - DEBUG - 24, 2019-06-12, Wednesday
06/07/2018 08:53:36 PM - root - DEBU

,date,week_index,weekday,week_number,month_number,month_name
0,2018-12-24,0,Monday,52,12,December
1,2018-12-25,0,Tuesday,52,12,December
2,2018-12-26,0,Wednesday,52,12,December
3,2018-12-27,0,Thursday,52,12,December
4,2018-12-28,0,Friday,52,12,December
5,2018-12-29,0,Saturday,52,12,December
6,2018-12-30,0,Sunday,52,12,December
7,2018-12-31,1,Monday,1,12,December
8,2019-01-01,1,Tuesday,1,1,January
9,2019-01-02,1,Wednesday,1,1,January


In [35]:
class Counters():
    def __init__(self):
        self.page = 0
        self.week = 0
    def reset(self):
        self.page = 0
        self.week = 0
        


In [36]:
counters = Counters()

In [37]:
def footer(canvas, s):
    canvas.drawString(width/2, 0, s)

In [38]:
def debug_print(x, y, s, canvas):
    if VERBOSITY == 0:
        pass
    elif VERBOSITY == 1:
        canvas.drawString(x, y, s)
    else:
        assert(False), f'Unknown value of VERBOSITY:{VERBOSITY}'
        

In [41]:
def gen_months_page(options, first_month, second_month, mycanvas):
    
    '''
    first_month is tuple (year, month)
    second_month is tuple (year, month)
    '''
    
    global counters
    
    logger.info(f"Generating months {first_month} and {second_month}")
    
    logger.info(f"Page number:{counters.page}")
    
    debug_print(0,height-10,"Months section", mycanvas)
    #mycanvas.drawString(0,height-10,"Months section")

    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    mycanvas.drawString(10,height-20,f"{calendar.month_name[first_month[1]]} {first_month[0]}")

    month_data = get_month(first_month[0],first_month[1])
    
    table = Table(month_data, colWidths=[20*mm,46*mm]) # altogether 66mm

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 10, 50)

    # ------- second month on a page
    
    month_data = get_month(second_month[0],second_month[1])

    mycanvas.drawString(250,height-20,f"{calendar.month_name[second_month[1]]} {second_month[0]}")

    table = Table(month_data, colWidths=[50,150])

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h2 = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 250, 50+h-h2) # use previous height h so that they are aligned at the top 
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() 
    counters.page += 1
    
if 1:
    VERBOSITY = 0
    counters.reset()
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
    gen_months_page(o, (2018,12), (2019,1), c)
    c.save()
    os.system("start " + 'test.pdf')

06/07/2018 08:53:53 PM - root - INFO - Generating months (2018, 12) and (2019, 1)
06/07/2018 08:53:53 PM - root - INFO - Page number:0


NameError: name 'get_month' is not defined

In [ ]:
from dateutil.rrule import rrule, MONTHLY

In [ ]:
def get_months_list(start_year, start_month, end_year, end_month):
    # return list of months
    # [(12, 2018), (1, 2019), (2, 2019), (3, 2019)]
    start = datetime.datetime(start_year, start_month, 1)
    end = datetime.datetime(end_year, end_month, 1)
    return [(d.year, d.month) for d in rrule(MONTHLY, dtstart=start, until=end)]

In [ ]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [ ]:
def gen_months_section(options, canvas):
    
    start_month = options.monthrange['start']
    last_month = options.monthrange['stop']
    
    logger.debug(start_month)
    logger.debug(last_month)
    
    number_of_months = diff_month(last_month, start_month) + 1
    
    logger.info(f"Number of months = {number_of_months}")
    
    assert(number_of_months>0), "Zero number of months is not allowed."
    
    if number_of_months % 2 == 0: #even number of months because there must be two on page
        logger.info("Even number of months.")
        pass
    else: # odd number of months, add one month
        logger.debug("Odd number of months, adding one month.")
        old_last_month = last_month
        last_month += monthdelta.monthdelta(1)
        logger.debug(f"{old_last_month} -> {last_month}")
    
    months = get_months_list(start_month.year, start_month.month, last_month.year, last_month.month)
    # e.g., [(12, 2018), (1, 2019), (2, 2019), (3, 2019)]
    
    logger.info(f"Months section generated for the following list of months:{months}")
    
    for i, m in enumerate(months[::2]): # iterate by two steps
        logger.debug(f"{i, i*2, i*2+1}, {m}")
        logger.debug(m) # first month on the page
        logger.debug(months[i*2+1]) # second month on the page
        gen_months_page(options, m, months[i*2+1], canvas)
    
if 1:
    logger.setLevel(logging.DEBUG)
    
    counters.reset()
    
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
   
    test_options = copy.deepcopy(o)
    test_options.monthrange['start'] = datetime.date(2018,5,15)
    test_options.monthrange['stop'] = datetime.date(2019,3,1)
    
    print(test_options)
    
    gen_months_section(test_options, c)
    c.save()
    os.system("start " + 'test.pdf')    
    

In [ ]:
cal = calendar.Calendar()
    
for i in calendar.Calendar().iterweekdays():
    print(i, calendar.day_name[i])

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
    

In [ ]:
def gen_week(options, week, mycanvas):
    ''' Generate a single week
    
    Current design is 1 week on 2 pages.
    
    week ... list of days in the week, first is Monday, last is Sunday
    [datetime.date(2018, 4, 30), datetime.date(2018, 5, 1), datetime.date(2018, 5, 2), 
     datetime.date(2018, 5, 3), datetime.date(2018, 5, 4), datetime.date(2018, 5, 5), datetime.date(2018, 5, 6)]
    
    week=
             date       week_index  weekday      week_number month_number month_name
        0  2018-12-24           0     Monday           52            12   December
        1  2018-12-25           0    Tuesday           52            12   December
        2  2018-12-26           0  Wednesday           52            12   December
        3  2018-12-27           0   Thursday           52            12   December
        4  2018-12-28           0     Friday           52            12   December
        5  2018-12-29           0   Saturday           52            12   December
        6  2018-12-30           0     Sunday           52            12   December
    
    '''
    
    global counters
    
    logger.debug(week)
    
    week_section_options = {
        'top_margin' : 9*mm,
        'bottom_margin' : 9*mm,
        'top_box_height' : 3*cm,
        'weekly_tasks_width' : 5*cm,
        'weekly_tasks_row_height' : 6*mm,
        'weekly_focus_height' : 5*cm,
        'day_box_width' : 7*cm,
        'day_box_nlines' : 4, # number of lines inside each day box 
    }
    
    debug_print(0,height-10,f"Week section, week={week.date.iloc[0]}-{week.date.iloc[-1]}", mycanvas)
    
    mycanvas.drawString(0, height-20, f"Week number:{week.week_number.iloc[0]}")
    
    # boxes for days of the week
    day_box_height =( height-week_section_options['top_margin']-week_section_options['bottom_margin'] ) / 7
        
    # spacing of lines inside each day box
    day_box_line_spacing = day_box_height / (week_section_options['day_box_nlines']+1)
    logger.debug(day_box_line_spacing)
        
    for i, (ind, day) in enumerate( week.iterrows() ):
        day_name = day['weekday']
        logger.debug(f"{day.date}:{day_name}")
        mycanvas.grid([week_section_options['weekly_tasks_width'], 
                       width],
                      [height-week_section_options['top_margin']-i*day_box_height, 
                       height-week_section_options['top_margin']-(i+1)*day_box_height])
        # name of the day in the day box
        mycanvas.drawString(week_section_options['weekly_tasks_width'], 
                            height-week_section_options['top_margin']-i*day_box_height-(1)*day_box_line_spacing+1*mm,
                            f"{day_name[0:2]}:{day.date.day}.{day.date.month}.")

                                                 
        mycanvas.setStrokeColor('pink')
        for j in range( week_section_options['day_box_nlines'] ):
            mycanvas.line(week_section_options['weekly_tasks_width'],
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          week_section_options['weekly_tasks_width'] + (width-week_section_options['weekly_tasks_width'])/2-0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
            mycanvas.line(week_section_options['weekly_tasks_width']+(width-week_section_options['weekly_tasks_width'])/2+0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          width,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
        mycanvas.setStrokeColor('black')
                      
    
    # top box
    
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['top_box_height'])
    
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, height-week_section_options['top_box_height'])
    
    
    # left column with week tasks
    
    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    
    nr = int( (height-week_section_options['top_box_height']-week_section_options['weekly_focus_height'])//(6*mm) )

    table = Table(data=[ [""] for i in range(int(nr))], 
                  colWidths=[5*cm], rowHeights=nr*[week_section_options['weekly_tasks_row_height']]) 
    
    table.setStyle(TableStyle(common_style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table height is {h}")
    table.drawOn(mycanvas, 0, week_section_options['weekly_focus_height'])
    
    
    # weekly focus subsection
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['weekly_focus_height'])
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0),
          (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, 0)
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() # complete page
    counters.page += 1
    
    return canvas
    
    

if 1:
    VERBOSITY = 1
    logger.setLevel(logging.DEBUG)
    counters.reset()
    
    days = get_days(o)
    weeks = days.groupby(by='week_index')
    
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
    
    week = weeks.get_group(0)
    print(week)
    gen_week(o, week, c)
    
    c.save()
    os.system("start " + 'test.pdf') 

In [ ]:
def gen_weeks(options, canvas):
    
    
    logger.info("Generating weeks")
    
    days = get_days(options)
    weeks = days.groupby(by='week_index')

    logger.info(f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}")
    
    debug_print(10,10,
                f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}",
                canvas)

    for w, week in weeks:
    
        logger.debug(f"week {w}")
        debug_print(10,20, f"week index={w},{week.week_index.iloc[0]}", canvas)
        gen_week(options, week, canvas)
        counters.week += 1

if 1:
    
    VERBOSITY = 1
    
    logger.setLevel(logging.DEBUG)
    
    counters.reset() # reset page counters
    
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
    
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    print(test_options)
    
    gen_weeks(test_options, c)

    c.save()
    os.system("start " + 'test.pdf') 
    


In [ ]:
def gen_weeks_withmonthly_notes(options, canvas):

    logger.info("Generating weeks with monthly notes")
    
    days = get_days(options)
    weeks = days.groupby(by='week_index')

    logger.info(f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}")
    
    debug_print(10,10,
                f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}",
                canvas)

    for w, week in weeks:
    
        logger.debug(f"week {w}")
        logger.debug(30*"=")
        logger.debug(f"week")
        debug_print(10,20, f"week index={w},{week.week_index.iloc[0]}", canvas)
        gen_week(options, week, canvas)
        counters.week += 1
        
        # Add notes pages at the end of each month, more precisely, add notes pages after the last week of each month
        # This is done by detecting that the last week had the month's last day in it
        
        last_week_first_day_month =  (week.date.iloc[0]).month # month to which the first day of the last week belongs
        logger.debug(f"First day of this week belongs to month {last_week_first_day_month}")
        
        last_day_of_month = datetime.date((week.date.iloc[0]).year, last_week_first_day_month, 
                                          calendar.monthrange((week.date.iloc[0]).year, last_week_first_day_month)[1])
        
        logger.debug([week.date==last_day_of_month])
        
        if (week.date==last_day_of_month).any():
            # add notes pages
            gen_notes(options, canvas)
            gen_notes(options, canvas)
            gen_notes(options, canvas)
        
if 1:
    
    VERBOSITY = 1
    
    logger.setLevel(logging.DEBUG)
    
    counters.reset() # reset page counters
    
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
    
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    print(test_options)
    
    gen_weeks_withmonthly_notes(test_options, c)

    c.save()
    os.system("start " + 'test.pdf')     

In [ ]:
def gen_notes(options, canvas):
    ''' Generate notes pages'''
    
    logger.info("Generating notes section")
    canvas.drawString(0,height-30,"debug:Notes section")
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray, None, (1,4)),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        #('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        #('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        #('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        #('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]
    
    # make as many rows as columns as to fill the page
    row_size = 9*mm
    col_size = 9*mm
    
    rows = int( (height-30) // row_size )
    logger.debug(f"Rows:{rows}")
    
    horizontal_margin = 30
    
    cols = int( (width-2*horizontal_margin) // col_size )
    logger.debug(f"Cols:{cols}")
    
    table_data = [ ['' for i in range(cols)] for i in range(rows) ]
    table = Table(table_data, colWidths=cols*[col_size], rowHeights=rows*[row_size])
    logger.debug(f"{rows}, {cols}")
    table.setStyle(TableStyle(style))
    
    w, h = table.wrapOn(canvas, 0, 0)
    table.drawOn(canvas, horizontal_margin, 0)
    
    footer(canvas, str(counters.page))
    canvas.showPage() 
    counters.page += 1

if 1:
    logger.setLevel(logging.DEBUG)
    
    counters.reset()
    
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)

    gen_notes(o, c)
    
    c.save()
    os.system("start " + 'test.pdf')    

In [ ]:
print(o)

In [ ]:
[a + ':' + str(o.__getattribute__(a)) for a in dir(o) if not a.startswith('__')]

In [ ]:
[a + ':' + str(o.__getattribute__(a)) for a in dir(o) if not a.startswith('__')]

In [ ]:
def gen_info_sheet(options, canvas):
    
    x = 10
    row_height = 20
    
    canvas.drawString(x,height-row_height,"INFO SHEET")
    
    i = 0
    for a in  dir(options):
        if a.startswith('__'):
            continue
        print(options.__getattribute__(a))
        canvas.drawString(x,height-(i+2)*row_height, \
            f"{a}::{str(options.__getattribute__(a))}")
        i += 1

    
    pass # no page number
    canvas.showPage()


if 1:
    c = canvas.Canvas('test.pdf', pagesize = o.pagesize)
    gen_info_sheet(o, c)
    c.save()
    os.system("start " + 'test.pdf')

# Generate calendar

In [ ]:

section_generators = {
    'info_sheet' : gen_info_sheet, 
    'months_v1'  : gen_months_section,
    'weeks_v1'   : gen_weeks,
    'weeks_v2'   : gen_weeks_withmonthly_notes,
    'notes_v1'   : gen_notes
}

logger.setLevel(logging.INFO)

counters.reset()

c = canvas.Canvas('diary.pdf', pagesize = o.pagesize)

for section in o.sections:
    logger.info(section)
    section_generators[section](o, c) # calls a generator as per dictionary
    
c.save()

os.system("start " + 'diary.pdf')